<a href="https://colab.research.google.com/github/ratllj/e-y-e-s/blob/main/collaboration_project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.4 MB/s eta 0:00:00


In [3]:
import os
import json
import numpy as np
from tqdm import tqdm
from pathlib import Path
from transformers import BertTokenizerFast
import tensorflow as tf


In [4]:
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_train.json -O KorQuAD_v1.0_train.json
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json -O KorQuAD_v1.0_dev.json

--2023-07-09 13:57:55--  https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38527475 (37M) [application/json]
Saving to: ‘KorQuAD_v1.0_train.json’

KorQuAD_v1.0_train. 100%[===================>]  36.74M   177MB/s    in 0.2s    

2023-07-09 13:57:56 (177 MB/s) - ‘KorQuAD_v1.0_train.json’ saved [38527475/38527475]

--2023-07-09 13:57:56--  https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3881058 (3.7M) [application/json]
Saving to: ‘KorQuAD_v1.0_dev.json’

KorQuAD_v1.0_dev.js 100%[======

In [5]:
def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return contexts, questions, answers

train_contexts, train_questions, train_answers = read_squad('KorQuAD_v1.0_train.json')
val_contexts, val_questions, val_answers = read_squad('KorQuAD_v1.0_dev.json')

In [6]:
print('훈련 데이터의 본문 개수 :', len(train_contexts))
print('훈련 데이터의 질문 개수 :', len(train_questions))
print('훈련 데이터의 답변 개수 :', len(train_answers))
print('테스트 데이터의 본문 개수 :', len(val_contexts))
print('테스트 데이터의 질문 개수 :', len(val_questions))
print('테스트 데이터의 답변 개수 :', len(val_answers))

훈련 데이터의 본문 개수 : 60407
훈련 데이터의 질문 개수 : 60407
훈련 데이터의 답변 개수 : 60407
테스트 데이터의 본문 개수 : 5774
테스트 데이터의 질문 개수 : 5774
테스트 데이터의 답변 개수 : 5774


In [7]:
def add_end_idx(answers, contexts):
  for answer, context in zip(answers, contexts):
    answer['text'] = answer['text'].rstrip()
    gold_text = answer['text']
    start_idx = answer['answer_start']
    end_idx = start_idx + len(gold_text)

    assert context[start_idx:end_idx] == gold_text, "end_index 계산에 에러 존재"
    answer['answer_end'] = end_idx

add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [8]:
tokenizer = BertTokenizerFast.from_pretrained('klue/bert-base')

train_encodings = tokenizer(train_contexts, train_questions, truncation = True, padding = True)
val_encodings = tokenizer(val_contexts, val_questions, truncation = True, padding = True)

In [9]:
def add_token_positions(encodings, answers):
  start_positions = []
  end_positions = []
  deleting_list = []

  for i in tqdm(range(len(answers))):
    start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
    end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']-1))

    if start_positions[-1] is None:
      start_positions[-1] = tokenizer.model_max_length
      deleting_list.append(i)

    if end_positions[-1] is None:
      end_positions[-1] = tokenizer.model_max_length
      if i not in deleting_list:
        deleting_list.append(i)

  encodings.update({'start_positions': start_positions, 'end_positions': end_positions})
  return deleting_list

In [10]:
deleting_list_for_train = add_token_positions(train_encodings, train_answers)
deleting_list_for_test = add_token_positions(val_encodings, val_answers)

100%|██████████| 5774/5774 [00:00<00:00, 304092.31it/s]


In [11]:
def delete_samples(encodings, deleting_list):
  input_ids = np.delete(np.array(encodings['input_ids']), deleting_list, axis=0)
  attention_masks = np.delete(np.array(encodings['attention_mask']), deleting_list, axis=0)
  start_positions = np.delete(np.array(encodings['start_positions']), deleting_list, axis = 0)
  end_positions = np.delete(np.array(encodings['end_positions']), deleting_list, axis = 0)

  X_data = [input_ids, attention_masks]
  y_data = [start_positions, end_positions]

  return X_data, y_data

In [12]:
X_train, y_train = delete_samples(train_encodings, deleting_list_for_train)
X_test, y_test = delete_samples(val_encodings, deleting_list_for_test)

In [13]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

In [14]:
strategy = tf.distribute.TPUStrategy(resolver)

In [16]:
class TFBertForQuestionAnswering(tf.keras.Model):
    def __init__(self, model_name):
        super(TFBertForQuestionAnswering, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.qa_outputs = tf.keras.layers.Dense(2,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                name='qa_outputs')
        self.softmax = tf.keras.layers.Activation(tf.keras.activations.softmax)

    def call(self, inputs):
        input_ids, attention_mask = inputs
        outputs = self.bert(input_ids, attention_mask=attention_mask)

        sequence_output = outputs[0]

        logits = self.qa_outputs(sequence_output)
        start_logits, end_logits = tf.split(logits, 2, axis=-1)

        # start_logits = (batch_size, sequence_length,)
        # end_logits = (batch_size, sequence_length,)
        start_logits = tf.squeeze(start_logits, axis=-1)
        end_logits = tf.squeeze(end_logits, axis=-1)

        start_probs = self.softmax(start_logits)
        end_probs = self.softmax(end_logits)

        return start_probs, end_probs

In [17]:
with strategy.scope():
  model = TFBertForQuestionAnswering("klue/bert-base")
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
  model.compile(optimizer=optimizer, loss=loss)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'bert.embeddings.position_ids', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [18]:
history = model.fit(
    X_train,
    y_train,
    epochs=3,
    verbose=1,
    batch_size=16,
)

Epoch 1/3


3759/3759 [==============================] - 528s 113ms/step - loss: 1.1283 - output_1_loss: 0.5068 - output_2_loss: 0.6215
Epoch 2/3
3759/3759 [==============================] - 430s 114ms/step - loss: 0.6199 - output_1_loss: 0.2648 - output_2_loss: 0.3551
Epoch 3/3
3759/3759 [==============================] - 427s 114ms/step - loss: 0.4440 - output_1_loss: 0.1822 - output_2_loss: 0.2618


In [19]:
loss_and_metrics = model.evaluate(X_test,y_test,batch_size=32)

print('##### Test Result #####')
print('loss : ',str(loss_and_metrics[0]))
print('Accuracy : ',str(loss_and_metrics[1]))


179/179 [==============================] - 26s 87ms/step - loss: 0.9955 - output_1_loss: 0.4512 - output_2_loss: 0.5444
##### Test Result #####
loss :  0.9955378174781799
Accuracy :  0.4511820077896118


In [24]:
def predict_test_data_by_idx(idx):
  context = tokenizer.decode(X_test[0][idx]).split('[SEP] ')[0]
  question = tokenizer.decode(X_test[0][idx]).split('[SEP] ')[1]
  print('본문 :', context)
  print('질문 :', question)
  answer_encoded = X_test[0][idx][y_test[0][idx]:y_test[1][idx]+1]
  print('정답 :',tokenizer.decode(answer_encoded))
  output = model([tf.constant(X_test[0][idx])[None, :], tf.constant(X_test[1][idx])[None, :]])
  start = tf.math.argmax(tf.squeeze(output[0]))
  end = tf.math.argmax(tf.squeeze(output[1]))+1
  answer_encoded = X_test[0][idx][start:end]
  print('예측 :',tokenizer.decode(answer_encoded))
  print('----------------------------------------')

In [27]:
for i in range(5000,5005):
  predict_test_data_by_idx(i)

본문 : [CLS] 1980년 봄, 그는 김대중, 김종필 등과 대권을 놓고 경쟁하였다. 그는 전두환과 신군부의 쿠데타를 그리 걱정하지 않았고, 결국 5 · 17 쿠데타로 좌절되었다. 1980년 5월 17일 오전 10시, 김영삼은 신군부 군인들에 의해 가택연금을 당했다. 김영삼은 5월 20일 상도동 자택에서 5. 17 비상계엄 확대 조치를 내린 신군부를 규탄하는 기자회견을 열었다. 그는 " 오늘 계엄통치를 확대 강화한 5 · 17 사태를, 민주회복이라는 국민적 목표를 배신한 폭거로 규정한다. 계엄당국의 강압통치로 빚어진 유혈사태는 이 나라를 파국으로 몰아가고있다. " 면서'국민적 목표를 배신한 5 · 17 폭거'라는 기자회견문을 발표했다. 이로 인해 김영삼은 신군부에 의해 5월 20일부터 가택연금 상태에 놓이게 되었다. 
질문 : 5. 17 비상계엄 확대 조치를 내린 신군부를 규탄하기 위해 김영삼이 발표한 기자회견문의 제목은? 
정답 : 국민적 목표를 배신한 5 · 17 폭거
예측 : 국민적 목표를 배신한 5 · 17 폭거
----------------------------------------
본문 : [CLS] 1980년 9월 출범한 전두환의 제5공화국 정권에서도 계속된 가택 연금과 정치적 탄압에 항의하며 장기간의 단식 투쟁을 단행하여 세계의 주목을 받았다. 같은해 10월 김영삼은 보안사 대공처장 이학봉의 강요로 정계 은퇴 선언을 발표하였다. 1981년 5월 연금에서 해제된 김영삼은 이민우 ( [UNK] [UNK] [UNK] ) · 김동영 ( 金 東 英 ) · 최형우 ( [UNK] [UNK] [UNK] ) · 김덕룡 ( 金 德 龍 ) 등 정치활동 규제에 묶여있는 재야 인사들과 함께 등산모임을 조직하고 민주산악회를 출범시켰다. 민주산악회의 참가자가 증가하면서 김영삼은 1981년 6월 9일 공식기구로서 출범하는데 동참하였다. 공식 기구로 출범한 민주산악회는 이민우를 회장으로 선출하고 김영삼을 고문으로 추대하였다. 그 뒤 민주산악회는 주요 정치적 사건에 대한 성명